# 코딩으로 확인하는 로지스틱 회귀

라이브러리 불러오기

In [56]:
import tensorflow as tf
import numpy as np

x, y값 정해주기

In [57]:
data = [[2,0], [4,0], [6,0], [8,1], [10,1], [12,1], [14,1]]
x_data = [x_row[0] for x_row in data]
y_data = [y_row[1] for y_row in data]

In [58]:
print("data = ", data)
print("x_data = ", x_data)
print("y_data = ", y_data)

data =  [[2, 0], [4, 0], [6, 0], [8, 1], [10, 1], [12, 1], [14, 1]]
x_data =  [2, 4, 6, 8, 10, 12, 14]
y_data =  [0, 0, 0, 1, 1, 1, 1]


a와 b의 값을 임의로 정하기

In [59]:
a = tf.Variable(tf.random_normal([1], dtype = tf.float64, seed = 0))
b = tf.Variable(tf.random_normal([1], dtype = tf.float64, seed = 0))

시그모이드 함수 방정식을 numpy 라이브러리를 이용해 다음과 같이 작성

$$y = \frac{1}{1+e^{(ax+b)}}$$

In [66]:
y = 1/(1 + np.e**(a * x_data + b))

오차를 구하는 함수 역시 numpy와 tensorflow를 이용해 다음과 같이 작성 (CROSS ENTROPY)

$$-(y\cdot \log h + (1-y)\log (1-h))$$

In [67]:
loss = -tf.reduce_mean(np.array(y_data) * tf.log(y) + (1-np.array(y_data)) *tf.log(1-y))

오차를 구하고 그 평균값을 loss 변수에 할당. 평균을 구하기 위해 reduce_mean() 함수를 사용.

이제 학습률을 지정하고 경사 하강법을 이용해 오차를 최소로 하는 값을 찾을 수 있음. **(LOSS를 최소로 하는 값 찾기)**

In [68]:
learning_rate = 0.5
gradient_decent = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

마지막으로 tensorflow를 구동시켜 결과값을 출력 **(학습)**

In [69]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(60001):
        sess.run(gradient_decent)
        if i % 6000 == 0:
            print("Epoch: %.f, loss = %.4f, 기울기 a = %.4f, y절편 = %.4f" % (i,sess.run(loss), sess.run(a), sess.run(b)))

Epoch: 0, loss = 1.2676, 기울기 a = 0.1849, y절편 = -0.4334
Epoch: 6000, loss = 0.0152, 기울기 a = -2.9211, y절편 = 20.2982
Epoch: 12000, loss = 0.0081, 기울기 a = -3.5637, y절편 = 24.8010
Epoch: 18000, loss = 0.0055, 기울기 a = -3.9557, y절편 = 27.5463
Epoch: 24000, loss = 0.0041, 기울기 a = -4.2380, y절편 = 29.5231
Epoch: 30000, loss = 0.0033, 기울기 a = -4.4586, y절편 = 31.0675
Epoch: 36000, loss = 0.0028, 기울기 a = -4.6396, y절편 = 32.3346
Epoch: 42000, loss = 0.0024, 기울기 a = -4.7930, y절편 = 33.4086
Epoch: 48000, loss = 0.0021, 기울기 a = -4.9261, y절편 = 34.3406
Epoch: 54000, loss = 0.0019, 기울기 a = -5.0436, y절편 = 35.1636
Epoch: 60000, loss = 0.0017, 기울기 a = -5.1489, y절편 = 35.9005


오차(loss)값이 점차 줄어들고 a와 b의 최적값을 찾아가는 것을 볼 수 있음.

# 여러 입력값을 갖는 로지스틱 회귀

변수가 많아지면 더 정확하게 예측을 할 수 있음. 변수를 하나 더 추가하여 x, y 데이터 만들기:

In [71]:
x_data = np.array([[2,3],[4,3],[6,4],[8,6],[10,7],[12,8],[14,9]])
y_data = np.array([0,0,0,1,1,1,1]).reshape(7,1)

In [72]:
print("x_data = ", x_data)
print("y_data = ", y_data)

x_data =  [[ 2  3]
 [ 4  3]
 [ 6  4]
 [ 8  6]
 [10  7]
 [12  8]
 [14  9]]
y_data =  [[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]


이제 tensorflow에서 데이터를 담는 **플레이스 홀더(placeholder)** 를 정해줌
TENSORFLOW 다이나믹
뒤에 데이터를 넣어줌.
그래프를 기반으로 돌기 때문에 최적화가 쉬움.

In [74]:
X = tf.placeholder(tf.float64, shape=[None,2])
Y = tf.placeholder(tf.float64, shape=[None,1])

변수가 $x$에서 $x_1$, $x_2$로 추가되면 각각의 기울기 $a_1$, $a_2$도 계산해야함. 즉 $ax$ 부분이 $a_1 x_1 + a_2 x_2$로 바뀜.

In [75]:
a = tf.Variable(tf.random_uniform([2,1], dtype = tf.float64))
b = tf.Variable(tf.random_uniform([1], dtype = tf.float64))

In [76]:
print("a = ", a)
print("b = ", b)

a =  <tf.Variable 'Variable_6:0' shape=(2, 1) dtype=float64_ref>
b =  <tf.Variable 'Variable_7:0' shape=(1,) dtype=float64_ref>


행렬곱을 이용해 $[a_1, a_2] * [x_1, x_2]$ 로 표현할 수 있음. Tensorflow에서는 **matmul()** 함수를 이용해 행렬곱을 적용함.

이번에는 **시그모이드**를 계산하기 위해 Tensorflow에 내장된 **sigmoid()** 함수를 사용.

In [77]:
y = tf.sigmoid(tf.matmul(X,a)+b) #시그모이드의 방정식

In [78]:
loss = -tf.reduce_mean(Y * tf.log(y) + (1-Y) * tf.log(1-y)) #오차를 구하는 함수

In [79]:
learning_rate = 0.1 #학습률 값

오차를 최소로 하는 값 찾기

In [80]:
gradient_decent = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [81]:
predicted = tf.cast(y>0.5, dtype = tf.float64)

In [82]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float64))

학습

In [83]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(3001):
        a_, b_, loss_, _ = sess.run([a,b,loss,gradient_decent], feed_dict={X: x_data, Y: y_data})
        if(i+1) % 300 == 0:
            print("step=%d, a1=%.4f, a2=%.4f, b=%.4f, loss=%.4f" % (i+1, a_[0], a_[1], b_, loss_))

step=300, a1=0.8780, a2=-0.6459, b=-2.4100, loss=0.2684
step=600, a1=0.8540, a2=-0.3426, b=-3.8679, loss=0.1934
step=900, a1=0.7550, a2=-0.0023, b=-4.9318, loss=0.1512
step=1200, a1=0.6462, a2=0.3117, b=-5.7756, loss=0.1237
step=1500, a1=0.5440, a2=0.5900, b=-6.4759, loss=0.1043
step=1800, a1=0.4522, a2=0.8347, b=-7.0748, loss=0.0901
step=2100, a1=0.3709, a2=1.0502, b=-7.5982, loss=0.0792
step=2400, a1=0.2993, a2=1.2411, b=-8.0631, loss=0.0705
step=2700, a1=0.2361, a2=1.4113, b=-8.4813, loss=0.0636
step=3000, a1=0.1799, a2=1.5643, b=-8.8614, loss=0.0579


오차값이 점차 줄어들고 $a_1, a_2, b$가 각각 최적값을 찾아가는 것이 보임

# 실제 값 적용하기

위에서 만든 다중 로지스틱 회귀 스크립트를 실제로 활용하여 예측값을 구해볼 수 있음.

[7시간 공부]하고 [과외를 6번] 받은 학생의 [합격 가능성]

In [86]:
new_x = np.array([7,6]).reshape(1,2)

In [88]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    new_y = sess.run(y, feed_dict={X: new_x})

In [89]:
new_y

array([[0.99945624]])